In [1]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F


In [2]:
mnist_data_train = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=T.ToTensor()
)

mnist_data_test = datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=T.ToTensor()
)

In [3]:
train_dataloader = DataLoader(mnist_data_train, batch_size=64, pin_memory=4, shuffle=True, )
test_dataloader = DataLoader(mnist_data_test, batch_size=64, pin_memory=4, shuffle=True, )

In [4]:
# class NeuraalNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.layer1 = nn.Linear(28 * 28, 128)
#         self.layer2 = nn.Linear(128, 128)
#         self.layer3 = nn.Linear(128, 10)
#         self.activation = nn.ReLU()
#         self.flatten = nn.Flatten()

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.activation(x)
#         x = self.layer2(x)
#         x = self.activation(x)
#         x = self.layer3(x)
#         x = self.activation(x)
#         return x

class CNN3Filters(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,  out_channels=6,  kernel_size=3)  # 28 → 26
        self.conv2 = nn.Conv2d(in_channels=6,  out_channels=16, kernel_size=3)  # 13 → 11
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)  # 5  → 3
        self.pool  = nn.MaxPool2d(kernel_size=2, stride=2)                      # /2 chaque fois

        self.fc1 = nn.Linear(32 * 1 * 1, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)      

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
device

device(type='mps')

In [6]:
model = CNN3Filters().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [7]:
writer = SummaryWriter()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train(train_loader: DataLoader, model: nn.Module, criterion: nn.Module, optimizer):
    model.train()
    total_loss = 0.0
    total_samples = 0
    correct_predictions = 0

    for X, y in train_loader:
        X = X.to(device).float()
        y = y.to(device)

        y_pred = model(X)
        loss = criterion(y_pred, y)
        batch_size = X.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size

        _, predicted = torch.max(y_pred, 1)
        correct_predictions += (predicted == y).sum().item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples * 100
    print(f"Average Loss: {average_loss:.4f}")
    print(f"Accuracy: {accuracy:.2f}%")

    writer.add_scalar('Loss/train', average_loss, epoch)
    writer.add_scalar('Accuracy/train', accuracy, epoch)

@torch.no_grad()
def test(model: nn.Module,
         dataloader: torch.utils.data.DataLoader,
         criterion: nn.Module,
         device: torch.device = torch.device("cpu")) -> tuple[float, float]:

    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device).float()
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    test_loss = running_loss / total
    accuracy = 100.0 * correct / total
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")
    return test_loss, accuracy

for epoch in range(50):
    train(train_dataloader, model, criterion, optimizer)
    test_loss, test_accuracy = test(model, test_dataloader, criterion, device)
    writer.add_scalar('Loss/test', test_loss, epoch)
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)
    print("Test Accuracy: {:.2f}%".format(test_accuracy))

writer.close()

model.eval()
with torch.no_grad():
    test_image, test_label = next(iter(test_dataloader))
    test_image = test_image[0:1].to(device)
    
    logits = model(test_image)
    probabilities = F.softmax(logits, dim=1)
    predicted = torch.argmax(logits, dim=1)
    
    print(f"Vraie étiquette: {test_label[0]}")
    print(f"Prédiction: {predicted[0]}")
    print(f"Logits bruts: {logits[0]}")
    print(f"Probabilités: {probabilities[0]}")

model_cpu = model.to('cpu')
dummy_input = torch.randn(1, 1, 28, 28)

torch.onnx.export(
    model_cpu,
    dummy_input,
    "mnist_cnn.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)

print("✅ Modèle ONNX exporté : mnist_cnn.onnx")

/Users/thomasfilhol/Documents/A3/iaweb/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Average Loss: 0.5008
Accuracy: 83.55%
Test Loss: 0.1801
Test Accuracy: 94.23%
Test Accuracy: 94.23%
Average Loss: 0.1620
Accuracy: 94.95%
Test Loss: 0.1264
Test Accuracy: 96.00%
Test Accuracy: 96.00%
Average Loss: 0.1200
Accuracy: 96.33%
Test Loss: 0.0937
Test Accuracy: 97.07%
Test Accuracy: 97.07%
Average Loss: 0.0981
Accuracy: 96.96%
Test Loss: 0.0882
Test Accuracy: 97.29%
Test Accuracy: 97.29%
Average Loss: 0.0867
Accuracy: 97.34%
Test Loss: 0.0757
Test Accuracy: 97.58%
Test Accuracy: 97.58%
Average Loss: 0.0755
Accuracy: 97.61%
Test Loss: 0.0691
Test Accuracy: 97.82%
Test Accuracy: 97.82%
Average Loss: 0.0667
Accuracy: 97.94%
Test Loss: 0.0597
Test Accuracy: 98.17%
Test Accuracy: 98.17%
Average Loss: 0.0602
Accuracy: 98.07%
Test Loss: 0.0724
Test Accuracy: 97.89%
Test Accuracy: 97.89%
Average Loss: 0.0561
Accuracy: 98.23%
Test Loss: 0.0615
Test Accuracy: 98.13%
Test Accuracy: 98.13%
Average Loss: 0.0531
Accuracy: 98.34%
Test Loss: 0.0641
Test Accuracy: 98.11%
Test Accuracy: 98.11%
